<a href="https://colab.research.google.com/github/HuaEdward/NBA-Predicting-Model/blob/main/BasketballGamePredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparing Data**

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("nba_games.csv", index_col = 0)

In [3]:
df

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
1,240.0,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,...,50.0,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False
2,240.0,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,...,20.0,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False
3,240.0,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,19.0,...,28.6,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True
4,240.0,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,20.0,...,16.8,30.9,157.0,90.0,MEM,92,1,2021,2021-04-30,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,240.0,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,...,34.2,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True
17768,240.0,240.0,37.0,74.0,0.500,13.0,25.0,0.520,26.0,37.0,...,25.0,30.0,139.0,129.0,ORL,108,1,2017,2016-12-14,True
17769,240.0,240.0,42.0,89.0,0.472,14.0,33.0,0.424,10.0,20.0,...,25.6,29.9,175.0,126.0,LAC,113,0,2017,2016-12-14,False
17770,240.0,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True


In [4]:
df = df.sort_values("date")

In [5]:
df = df.reset_index(drop=True)

In [6]:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

In [7]:
def add_target(team):
  team["target"] = team["won"].shift(-1)
  return team

df = df.groupby("team", group_keys=False).apply(add_target)

<ipython-input-7-9d822698de18>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
<ipython-input-7-9d822698de18>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
<ipython-input-7-9d822698de18>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe =

In [8]:
df["target"][pd.isnull(df["target"])] = 2

<ipython-input-8-2c3f464000fb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"][pd.isnull(df["target"])] = 2


In [9]:
df["target"] = df["target"].astype(int, errors="ignore")

In [10]:
nulls = pd.isnull(df)

In [11]:
nulls = nulls.sum()

In [12]:
nulls = nulls[nulls > 0]

In [13]:
nulls

,0
+/-,17772
mp_max,17772
mp_max.1,17772
+/-_opp,17772
mp_max_opp,17772
mp_max_opp.1,17772


In [14]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [15]:
valid_columns

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%',
       ...
       'usg%_max_opp', 'ortg_max_opp', 'drtg_max_opp', 'team_opp', 'total_opp',
       'home_opp', 'season', 'date', 'won', 'target'],
      dtype='object', length=142)

In [16]:
df = df[valid_columns].copy()

In [17]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,0.704,...,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False,0
1,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,...,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False,1
2,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,...,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True,1
3,240.0,41.0,96.0,0.427,9.0,30.0,0.300,20.0,22.0,0.909,...,38.9,201.0,120.0,NOP,95,0,2016,2015-10-27,True,1
4,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,...,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,240.0,34.0,85.0,0.400,15.0,38.0,0.395,14.0,19.0,0.737,...,36.3,133.0,112.0,GSW,107,0,2022,2022-06-10,False,0
17768,240.0,41.0,88.0,0.466,9.0,40.0,0.225,13.0,15.0,0.867,...,94.4,300.0,112.0,BOS,94,0,2022,2022-06-13,True,1
17769,240.0,31.0,75.0,0.413,11.0,32.0,0.344,21.0,31.0,0.677,...,36.2,222.0,107.0,GSW,104,1,2022,2022-06-13,False,0
17770,240.0,34.0,80.0,0.425,11.0,28.0,0.393,11.0,12.0,0.917,...,31.5,186.0,111.0,GSW,103,0,2022,2022-06-16,False,2


# **Selecting Features**

In [18]:
%pip install scikit-learn

In [19]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)
sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction="forward", cv=split)

In [20]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]

In [21]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [22]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [23]:
sfs.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [24]:
predictors = list(selected_columns[sfs.get_support()])

# **Creating Baseline Model**

In [25]:
def backtest(data, model, predictors, start=2, step=1):
  all_predictions = []

  seasons = sorted(data["season"].unique())
  for i in range(start, len(seasons), step):
    season = seasons[i]

    train = data[data["season"] < season]
    test = data[data["season"] == season]

    model.fit(train[predictors], train["target"])

    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index)

    combined = pd.concat([test["target"], preds], axis=1)
    combined.columns = ["actual", "prediction"]

    all_predictions.append(combined)
  return pd.concat(all_predictions)

In [26]:
predictions = backtest(df, rr, predictors)

In [27]:
from sklearn.metrics import accuracy_score

In [28]:
predictions = predictions[predictions["actual"] != 2]
accuracy_score(predictions["actual"], predictions["prediction"])

0.5485110470701249

In [29]:
df.groupby("home").apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])

,0
home,
0.0,0.428314
1.0,0.571686


# **Improving performence**
(with rolling average)

In [30]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

In [31]:
import numpy as np

In [32]:
df_rolling = df[list(selected_columns) + ["won", "season", "team"]]

def find_team_averages(team):
    numeric_columns = team.select_dtypes(include=[np.number]).columns
    rolling = team[numeric_columns].rolling(10).mean()
    rolling["team"] = team["team"]
    rolling["season"] = team["season"]
    return rolling

df_rolling = df.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

In [33]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols

df = pd.concat([df, df_rolling], axis=1)

In [34]:
df = df.dropna()

# **Adding in opponent info**

In [35]:
def shift_col(team, col_name):
  next_col = team[col_name].shift(-1)
  return next_col

def add_col(df, col_name):
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

In [36]:
df = df.copy()

In [37]:
full = df.merge(
    df[rolling_cols + ["team_opp_next", "date_next", "team"]],
    left_on=["team", "date_next"],
    right_on=["team_opp_next", "date_next"]
)

In [38]:
full[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]]

,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
0,SAC,TOR,TOR,SAC,2015-11-15
1,TOR,SAC,SAC,TOR,2015-11-15
2,CLE,DET,DET,CLE,2015-11-17
3,GSW,TOR,TOR,GSW,2015-11-17
4,DEN,NOP,NOP,DEN,2015-11-17
...,...,...,...,...,...
15769,BOS,GSW,GSW,BOS,2022-06-10
15770,GSW,BOS,BOS,GSW,2022-06-13
15771,BOS,GSW,GSW,BOS,2022-06-13
15772,GSW,BOS,BOS,GSW,2022-06-16


# **Training the final model**

In [39]:
removed_columns = list(full.columns[full.dtypes == object]) + removed_columns

In [40]:
removed_columns

['team_x',
 'team_opp',
 'date',
 'team_10_x',
 'team_opp_next_x',
 'date_next',
 'team_10_y',
 'team_opp_next_y',
 'team_y',
 'season',
 'date',
 'won',
 'target',
 'team',
 'team_opp']

In [41]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]

In [42]:
sfs.fit(full[selected_columns], full["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [43]:
predictors = list(selected_columns[sfs.get_support()])

In [44]:
predictors

['blk%',
 'usg%',
 'trb_max',
 'ftr_max',
 'orb%_max',
 'usg%_max',
 'orb_opp',
 'tov_opp',
 'usg%_opp',
 'fga_max_opp',
 'ast_max_opp',
 'fg_10_x',
 'usg%_10_x',
 '+/-_max_10_x',
 'usg%_opp_10_x',
 'orb_max_opp_10_x',
 '+/-_max_opp_10_x',
 'target_10_x',
 'home_next',
 'ts%_10_y',
 'usg%_10_y',
 '3p_max_10_y',
 'ortg_max_10_y',
 'fga_opp_10_y',
 'pts_opp_10_y',
 'usg%_opp_10_y',
 '+/-_max_opp_10_y',
 'ts%_max_opp_10_y',
 'ortg_max_opp_10_y',
 'target_10_y']

In [45]:
predictions = backtest(full, rr, predictors)

In [46]:
accuracy_score(predictions["actual"], predictions["prediction"])

0.7812021267009102